<a href="https://colab.research.google.com/github/ddPn08/tensorrt-diffusion-colab/blob/main/tensorrt_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Mount Google Drive
mount_gdrive = True  # @param{type:"boolean"}

if mount_gdrive:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=False)

In [ ]:
# @title Install dependencies

import os

! curl -LO https://github.com/ddPn08/tensorrt-diffusion-colab/releases/download/tensorrt-8.5.2/libnvinfer_plugin.so.8
! apt update && apt install software-properties-common -y && add-apt-repository --yes ppa:deadsnakes/ppa
! apt update && apt install tensorrt tensorrt-dev tensorrt-libs git-lfs -y
! git lfs install
! git clone https://github.com/NVIDIA/TensorRT TensorRT

TRT_OSSPATH = os.path.abspath("./TensorRT")
script_filepath = os.path.join(TRT_OSSPATH, "demo", "Diffusion", "build.py")

! curl -o {script_filepath} -L https://raw.githubusercontent.com/ddPn08/tensorrt-diffusion-colab/main/scripts/build.py
! pip install --upgrade pip
! cd TensorRT/demo/Diffusion && pip install -r requirements.txt && pip install tensorrt accelerate numpy==1.21.6

def make_args(d):
    arguments = []
    for k, v in d.items():
        if type(v) == bool:
            arguments.append(f"--{k}" if v else "")
            continue
        if type(v) == str and v:
            arguments.extend([f"--{k}", f"\"{v}\""])
            continue
        if type(v) == int or type(v) == float:
            arguments.extend([f"--{k}", f"{v}"])
            continue
        if v:
            arguments.extend([f"--{k}", f"{v}"])
            continue
    return " ".join(arguments)

# Build tensorrt engine
Convert the Diffusers model to TensorRT format.

The converted model depends on the converted environment, so models with different GPUs may not be available.

In [ ]:
# @title Build
import os

def make_args(d):
    arguments = []
    for k, v in d.items():
        k = k.replace("_", "-")
        if type(v) == bool:
            arguments.append(f"--{k}" if v else "")
        elif type(v) == str and v:
            arguments.extend([f"--{k}", f"{v}"])
        elif v:
            arguments.extend([f"--{k}", f"{v}"])
    return " ".join(arguments)

unet_pretrained_model_id = "JosephusCheung/ACertainThing"  # @param{type:"string"}
vae_pretrained_model_id = "JosephusCheung/ACertainThing"  # @param{type:"string"}
clip_pretrained_model_id = "openai/clip-vit-large-patch14"  # @param{type:"string"}

os.environ["UNET_PRETRAINED_MODEL_ID"] = unet_pretrained_model_id
os.environ["VAE_PRETRAINED_MODEL_ID"] = vae_pretrained_model_id
os.environ["CLIP_PRETRAINED_MODEL_ID"] = clip_pretrained_model_id

# @markdown <br>

engine_dir = "/content/engines/ACertainThing"  # @param{type:"string"}
onnx_dir = "/content/onnx/ACertainThing"  # @param{type:"string"}

os.makedirs(engine_dir, exist_ok=True)
os.makedirs(onnx_dir, exist_ok=True)

# @markdown <br>

hf_token = ""  # @param{type:"string"}
denoising_prec = "fp32"  # @param ["fp32", "fp16"]
scheduler = "LMSD"  # @param ["LMSD", "DPM"]
height = 512  # @param{type:"slider", min:256, max:1024, step:64}
width = 512  # @param{type:"slider", min:256, max:1024, step:64}

onnx_opset = 16  # @param{type:"integer"}
force_onnx_export = False  # @param{type:"boolean"}
force_onnx_optimize = False  # @param{type:"boolean"}
onnx_minimal_optimization = False  # @param{type:"boolean"}

force_engine_build = False  # @param{type:"boolean"}
build_static_batch = False  # @param{type:"boolean"}
build_dynamic_shape = False  # @param{type:"boolean"}
build_preview_features = False  # @param{type:"boolean"}

args = make_args({
    "engine-dir": engine_dir,
    "onnx-dir": onnx_dir,
    "hf-token": hf_token,
    "denoising-prec": denoising_prec,
    "scheduler": scheduler,
    "height": height,
    "width": width,
    "onnx-opset": onnx_opset,
    "force-onnx-export": force_onnx_export,
    "force-onnx-optimize": force_onnx_optimize,
    "onnx-minimal-optimization": onnx_minimal_optimization,
    "force-engine-build": force_engine_build,
    "build-static-batch": build_static_batch,
    "build-dynamic-shape": build_dynamic_shape,
    "build-preview-features": build_preview_features
})

! cd /content/TensorRT/demo/Diffusion && LD_PRELOAD="/content/libnvinfer_plugin.so.8" python {script_filepath} {args}

## Upload built engine to huggingface

In [ ]:
# @title Login to your Hugface account
from huggingface_hub import login

hf_token = ""  # @param{type:"string"}
login(token=hf_token, add_to_git_credential=True)

In [ ]:
# @title Upload built model to huggingface
from huggingface_hub import create_repo, HfApi

api = HfApi()

user_name = ""  # @param{type:"string"}
repository_name = ""  # @param{type:"string"}
private = False  # @param{type:"boolean"}

repo_id = f"{user_name}/{repository_name}"

create_repo(repo_id, repo_type=repo_type, private=private)

api.upload_folder(
    folder_path=engine_dir,
    path_in_repo="engine",
    repo_id=repo_id,
    repo_type="model"
)
api.upload_folder(
    folder_path=onnx_dir,
    path_in_repo="onnx",
    repo_id=repo_id,
    repo_type="model"
)

# Inference with the tensorrt engine
Use pre-built engines for inference.

In [ ]:
import os
engines_dir = "/content/engines"
engine_repository = "ddPn08/ACertainThing-colab-tensorrt"  # @param["https://huggingface.co/ddPn08/stable-diffusion-v1.4-colab-tensorrt", "ddPn08/ACertainThing-colab-tensorrt"] {"allow-input": true}
engine_repository_name = engine_repository.split("/")[-1]

url = f"https://huggingface.co/{engine_repository}"

os.makedirs(engines_dir, exist_ok=True)

! cd {engines_dir} && git clone {url}

In [ ]:
import os

engine_dir = "/content/engines/ACertainThing-colab-tensorrt/engine"  # @param{type:"string"}
output_dir = "/content/outputs"  # @param{type:"string"}
hf_token = ""  # @param{type:"string"}

os.makedirs(output_dir, exist_ok=True)

# @markdown ---

prompt = "masterpiece, best quality, 1girl"  # @param{type:"string"}
negative_prompt = "worst quality, low quality, deleted, lowres, bad anatomy, bad hands, text, error, missing fingers, extra digits, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry"  # @param{type:"string"}
repeat_prompt = 1  # @param{type:"integer"}
height = 512  # @param{type:"slider", min:256, max:1024, step:64}
width = 512  # @param{type:"slider", min:256, max:1024, step:64}
denoising_steps = 150  # @param{type:"integer"}
denoising_prec = "fp32"  # @param ["fp32", "fp16"]
scheduler = "LMSD"  # @param ["LMSD", "DPM"]
seed = "1"  # @param{type:"string"}

num_warmup_runs = 0  # @param{type:"integer"}

args = make_args({
    "engine-dir": engine_dir,
    "output-dir": output_dir,
    "negative-prompt": negative_prompt,
    "repeat-prompt": repeat_prompt,
    "height": height,
    "width": height,
    "denoising-steps": denoising_steps,
    "denoising-prec": denoising_prec,
    "scheduler": scheduler,
    "seed": seed,
    "num-warmup-runs": num_warmup_runs
})

! cd /content/TensorRT/demo/Diffusion \
    && LD_PRELOAD="/content/libnvinfer_plugin.so.8" accelerate launch demo-diffusion.py "{prompt}" {args}